# Preprocess USMLE for finetuning and evaluation

## Create version for finetuning

In [1]:
from datasets import load_dataset

gcs_bucket_name = "open-llm-finetuning"

# Load model
usml_raw = load_dataset("GBaker/MedQA-USMLE-4-options")
usml_train = usml_raw['train']

print(usml_raw)

/Users/joshhaber/SIADS-699_PEFT-Testers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})


In [ ]:
# Preprocess dataset for finetuning
def format_mcf_finetuning(example):
    option_keys = sorted(example['options'].keys())
    formatted_options = "\n".join([f"{key}. {example['options'][key]}" for key in option_keys])
    prompt = f"Question: {example['question']}\n{formatted_options}\nAnswer:"
    completion = example['answer']
    return {"prompt": prompt, "completion": completion}

usml_train_data = usml_train.map(
    format_mcf_finetuning,
    remove_columns=usml_train.column_names
)

# Save
#usml_train_data.save_to_disk(f"gcs://{gcs_bucket_name}/data/finetuning/train_usml")

In [3]:
print(usml_train_data[0]['prompt'])
print(usml_train_data[0]['completion'])

Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
A. Ampicillin
B. Ceftriaxone
C. Doxycycline
D. Nitrofurantoin
Answer:
Nitrofurantoin


## Version for evaluation

In [5]:
usml_eval = usml_raw['test']

usml_eval_data = usml_eval.map(
    format_mcf_finetuning,
    remove_columns=usml_eval.column_names
)

# Save
#usml_eval_data.save_to_disk(f"gcs://{gcs_bucket_name}/data/finetuning/eval_usml")

## Preprocess Prompt and Answer Letter for Completion

In [6]:
# Preprocess dataset for finetuning
def format_letter_finetuning(example):
    option_keys = sorted(example['options'].keys())
    formatted_options = "\n".join([f"{key}. {example['options'][key]}" for key in option_keys])
    prompt = f"Question: {example['question']}\n{formatted_options}\nAnswer:"
    completion = example['answer_idx']
    return {"prompt": prompt, "completion": completion}

usml_train_letter_data = usml_train.map(
    format_letter_finetuning,
    remove_columns=usml_train.column_names
)

print(usml_train_letter_data[0]['prompt'])
print(usml_train_letter_data[0]['completion'])

Map: 100%|██████████| 10178/10178 [00:00<00:00, 27516.80 examples/s]

Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
A. Ampicillin
B. Ceftriaxone
C. Doxycycline
D. Nitrofurantoin
Answer:
D
